# FAISS 인덱스 구조 및 저장 방식
FAISS 벡터 데이터베이스의 개념과 저장 방식을 학습합니다.
1. FAISS는 왜 필요한가? (간단 복습)
2. 저장된 FAISS 인덱스는 어떻게 구성되어 있을까?
3. FAISS 인덱스 불러오기 및 검색 실습
4. 정리 및 다음 실습 안내

### 1. FAISS는 왜 필요한가?

앞 실습에서는 텍스트를 벡터로 변환한 뒤, `FAISS`를 통해 인덱스를 생성하고 저장했습니다.

```python
faiss_index = FAISS.from_texts(texts=chunks, embedding=embedding)
faiss_index.save_local("faiss_index_e5")
````

그렇다면 저장된 이 인덱스는 **어떤 구조로 구성되어 있고**,
**어떻게 다시 불러와서 사용할 수 있을까요?**
이번 시간엔 그 구조와 재사용 방식까지 함께 알아봅니다.


### 2. 저장된 FAISS 인덱스 구조 살펴보기

`faiss_index.save_local("faiss_index_sample")` 명령을 실행하면
다음과 같은 파일들이 생성됩니다:

```
faiss_index_sample/
├── index.faiss   ← 벡터 인덱스가 저장된 바이너리
└── index.pkl     ← 텍스트 내용 등 메타 정보 (Python 객체)
```

* **`index.faiss`** : FAISS 라이브러리가 생성한 실제 벡터 인덱스 데이터가 저장된 바이너리 파일입니다.
* **`index.pkl`** : LangChain에서 관리하는 문서 내용 및 기타 메타데이터가 포함된 Python 객체입니다.

> 이 두 파일을 함께 저장하고 불러와야, 검색 시 원래 문서 내용까지 확인할 수 있습니다.

### 3. 저장된 인덱스 불러와서 검색해보기

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# 동일한 임베딩 모델을 로딩해야 로드 가능
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

# 저장된 인덱스 불러오기
faiss_index = FAISS.load_local(
    "embeddings/faiss_index_e5",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)

# 검색기 준비
retriever = faiss_index.as_retriever(search_kwargs={"k": 3})

# 테스트 쿼리
query = "국회의 임기"
results = retriever.get_relevant_documents(query)
for i, doc in enumerate(results, 1):
    print(f"{i}.", doc.page_content.strip())

1. 제86조 ①국무총리는 국회의 동의를 얻어 대통령이 임명한다.
2. 제104조 ①대법원장은 국회의 동의를 얻어 대통령이 임명한다.
3. ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다.


### 실습 시 주의점

* `load_local()` 시에는 **동일한 임베딩 모델**을 다시 불러와야 합니다.
* `k`값을 조정하면 검색 결과 개수를 제어할 수 있습니다.

### 정리

* FAISS는 벡터 인덱스를 `.faiss`와 `.pkl` 두 개의 파일로 구성합니다.
* `save_local()` / `load_local()`을 통해 저장과 복원이 매우 간단합니다.
* RAG 파이프라인에서는 **벡터 재사용이 가능하도록 구조화된 저장 방식**이 핵심입니다.